# Autoloading

Loading graph visualisation settings.

In [ ]:
%%graph_notebook_vis_options
{
  "edges": {
    "smooth": {
      "enabled": true,
      "type": "dynamic"
    },
    "arrows": {
      "to": {
        "enabled": true,
        "type": "arrow"
      }
    }
  }
}

# Initial Setup

## Get a view of all Ingested Cluster

Retrieve all the current cluster ingested in KubeHound with the associated runID with the number of nodes. This numbers can be used to get a clue of the size of the cluster and also identify if an ingestion did not complete.

In [14]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.nodes()
    .groupCount()
    .by(project('cluster','runID')
         .by('cluster').by('runID'))
    .unfold()
    .limit(100)  // Limit the number of results for large clusters

## Setting your run_id/cluster

Set which runID you want to use. The variable are being shared with the user, so we advise to make a uniq string for your user `runID_yourid` to avoid any conflict.

In [11]:
%%gremlin -d class -g critical -le 50 -p inv,oute

graph.variables()
    .set('runID_yourid',' 01htdgjj34mcmrrksw4bjy2e94')

# Container escapes

List all containers which are vulnerable to container escape to the node. 

## Identify the vulnerable containers

The goal of this list is to identify which image is being vulnerable to container escape. It will list all the containers with `namespace`, `app`, `team` and `image`.

In [12]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get())
    .where(
        repeat(
          outE().inV().simplePath()  // Building the path
        ).until(
            has(label, "Node")       // Stop when meeting a critical asset
            .or().loops().is(10)     // Stop after 4 iteration
        ).has(label, "Node")         // Keep only path ending with a critical asset
        .limit(1)
    )
    .dedup().by("image")
    .valueMap("namespace","app","team","image")
    .limit(1000)

The following list give a more abstract view to get deduplicated list of vulnerable `app`/`namespace`.

The goal here is to extact a list of apps to whitelist. If the flags are not set properly, you scope it by `namespace`.

In [13]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get())
    .where(
        repeat(
          outE().inV().simplePath()  // Building the path
        ).until(
            has(label, "Node")       // Stop when meeting a critical asset
            .or().loops().is(10)     // Stop after 4 iteration
        ).has(label, "Node")         // Keep only path ending with a critical asset
        .limit(1)
    )
    .by("namespace")
    .by("app")
    .valueMap("namespace","app")
    .limit(1000)

## Manual investigation for each app/namespace

From the above list, you can iterate manual investigation by scoping by each vulnerable `app`/`namespace`. To process the investigation, just copy/paste the name of the vulnerable app (replace `VULNERABLE_APP` by the targetted app).

### Listing all attack paths from a particular app

The following gremlin request will **list all container escapes**. We add a limit(1000) to avoid having huge graph.

In [14]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get())
    .has("app","VULNERABLE_APP")
    .repeat(
      outE().inV().simplePath()  // Building the path
    ).until(
        has(label, "Node")    // Stop when meeting a critical asset
        .or().loops().is(10)      // Stop after 4 iteration
    ).has(label, "Node")       // Keep only path ending with a critical asset
    .path().by(elementMap())
    .limit(1000)

The last view can be quite overwhelming. Also the last view can be capped with the limit(1000), so we dont have an exhaustive view. Increasing the limit will not solve the issue as, it will become non human readable.  

### Listing all attack path deduplicated by app from a particular app 

One wau to solve it is to general an **overall view to understand the attack path**. This view will strip any specific information (image, ids, ...). For instance, this will remove any replicatset duplication.

You need to replace `VULNERABLE_APP` by the targetted app.

In [4]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get())
    .has("app","VULNERABLE_APP")
    .repeat(
      outE().inV().simplePath()  // Building the path
    ).until(
        has(label, "Node")    // Stop when meeting a critical asset
        .or().loops().is(10)      // Stop after 4 iteration
    ).has(label, "Node")       // Keep only path ending with a critical asset
    .path()
    .by(valueMap("app", "class","critical").with(WithOptions.tokens,WithOptions.labels))
    .dedup()
    .limit(1000)  // Limit the number of results for large clusters

Something, this view is still too big and return too many elements to be easily processable. 

### Listing all attack path deduplicated by label/type from a particular app 

To get an even widder picture, we can deduplicate the attack paths by label. This show the interaction from one type (endpoints/containers/nodes/...) to try to understand the bigger picture.

You need to replace `VULNERABLE_APP` by the targetted app.

In [18]:
%%gremlin -d class -g critical -le 50 -p inv,oute

kh.containers()
    .has("runID", graph.variables().get('runID_yourid').get())
    .has("app","VULNERABLE_APP")
    .repeat(
      outE().inV().simplePath()  // Building the path
    ).until(
        has(label, "Node")    // Stop when meeting a critical asset
        .or().loops().is(10)      // Stop after 4 iteration
    ).has(label, "Node")       // Keep only path ending with a critical asset
    .path().by(label())
    .dedup()
    .limit(1000)